In [5]:
import pandas as pd
import requests
import json
from pprint import pprint

In [2]:
#base url
base_url = 'https://fantasy.premierleague.com/api/'

r = requests.get(base_url +'bootstrap-static/').json()

In [9]:
# show the top level fields
pprint(r, indent=2, depth=1, compact=True)

{ 'element_stats': [...],
  'element_types': [...],
  'elements': [...],
  'events': [...],
  'game_settings': {...},
  'phases': [...],
  'teams': [...],
  'total_players': 10997846}


# Player data

The elements field contains data for each premier league player in the current season of FPL. We can access these data just like we would access data associated with a key in a dictionary. The response is a list of more dictionaries — one per player.

Let’s get the elements data and then show the information about the first player in the list:

In [13]:
players = r['elements']
pprint((players[4]))

In [16]:
pd.set_option('display.max_columns', None)

#create players dataframe 
players = pd.json_normalize(r['elements'])

In [18]:
players[['id', 'web_name', 'team', 'element_type']].head()

,id,web_name,team,element_type
0,1,Cédric,1,2
1,3,Xhaka,1,3
2,4,Elneny,1,3
3,5,Holding,1,2
4,6,Partey,1,3


# Supporting data 

We can get the names and strength ratings for teams by extracting the teams field from the base response into a dataframe:

In [19]:
#create teams dataframe 

teams = pd.json_normalize(r['teams'])

In [20]:
teams

,code,draw,form,id,loss,name,played,points,position,short_name,strength,team_division,unavailable,win,strength_overall_home,strength_overall_away,strength_attack_home,strength_attack_away,strength_defence_home,strength_defence_away,pulse_id
0,3,0,None,1,0,Arsenal,0,0,0,ARS,4,None,False,0,1220,1270,1240,1250,1200,1270,1
1,7,0,None,2,0,Aston Villa,0,0,0,AVL,3,None,False,0,1090,1100,1110,1130,1090,1110,2
2,91,0,None,3,0,Bournemouth,0,0,0,BOU,3,None,False,0,1060,1090,1070,1130,1050,1080,127
3,94,0,None,4,0,Brentford,0,0,0,BRE,3,None,False,0,1100,1130,1100,1110,1130,1160,130
4,36,0,None,5,0,Brighton,0,0,0,BHA,3,None,False,0,1150,1160,1140,1160,1170,1190,131
5,8,0,None,6,0,Chelsea,0,0,0,CHE,4,None,False,0,1200,1220,1200,1210,1180,1210,4
6,31,0,None,7,0,Crystal Palace,0,0,0,CRY,3,None,False,0,1100,1140,1100,1170,1080,1140,6
7,11,0,None,8,0,Everton,0,0,0,EVE,3,None,False,0,1080,1100,1080,1080,1080,1080,7
8,54,0,None,9,0,Fulham,0,0,0,FUL,3,None,False,0,1075,1100,1070,1070,1100,1130,34
9,13,0,None,10,0,Leicester,0,0,0,LEI,3,None,False,0,1130,1130,1080,1160,1180,1120,26


# Player Positions 


In [23]:
positions = pd.json_normalize(r['element_types'])

In [24]:
positions

,id,plural_name,plural_name_short,singular_name,singular_name_short,squad_select,squad_min_play,squad_max_play,ui_shirt_specific,sub_positions_locked,element_count
0,1,Goalkeepers,GKP,Goalkeeper,GKP,2,1,1,True,[12],72
1,2,Defenders,DEF,Defender,DEF,5,3,5,False,[],236
2,3,Midfielders,MID,Midfielder,MID,5,2,5,False,[],288
3,4,Forwards,FWD,Forward,FWD,3,1,3,False,[],79


# Combining all data to get final player dataframe

In [25]:
# join players to teams
df = pd.merge(
    left=players,
    right=teams,
    left_on='team',
    right_on='id'
)

In [30]:
# join player positions
df = df.merge(
    positions,
    left_on='element_type',
    right_on='id'
)
# rename columns
df = df.rename(
    columns={'name':'team_name', 'singular_name':'position_name'}
)
# show result
df

<ipython-input-30-18f0d182c0a6>:2: FutureWarning: Passing 'suffixes' which cause duplicate columns {'id_x'} in the result is deprecated and will raise a MergeError in a future version.
  df = df.merge(


,chance_of_playing_next_round,chance_of_playing_this_round,code_x,cost_change_event,cost_change_event_fall,cost_change_start,cost_change_start_fall,dreamteam_count,element_type,ep_next,ep_this,event_points,first_name,form_x,id_x,in_dreamteam,news,news_added,now_cost,photo,points_per_game,second_name,selected_by_percent,special,squad_number,status,team,team_code,total_points,transfers_in,transfers_in_event,transfers_out,transfers_out_event,value_form,value_season,web_name,minutes,goals_scored,assists,clean_sheets,goals_conceded,own_goals,penalties_saved,penalties_missed,yellow_cards,red_cards,saves,bonus,bps,influence,creativity,threat,ict_index,starts,expected_goals,expected_assists,expected_goal_involvements,expected_goals_conceded,influence_rank,influence_rank_type,creativity_rank,creativity_rank_type,threat_rank,threat_rank_type,ict_index_rank,ict_index_rank_type,corners_and_indirect_freekicks_order,corners_and_indirect_freekicks_text,direct_freekicks_order,direct_freekicks_text,penalties_order,penalties_text,expected_goals_per_90,saves_per_90,expected_assists_per_90,expected_goal_involvements_per_90,expected_goals_conceded_per_90,goals_conceded_per_90,now_cost_rank,now_cost_rank_type,form_rank,form_rank_type,points_per_game_rank,points_per_game_rank_type,selected_rank,selected_rank_type,starts_per_90,clean_sheets_per_90,code_y,draw,form_y,id_y,loss,team_name,played,points,position,short_name,strength,team_division,unavailable,win,strength_overall_home,strength_overall_away,strength_attack_home,strength_attack_away,strength_defence_home,strength_defence_away,pulse_id,id_x,plural_name_x,plural_name_short_x,position_name,singular_name_short_x,squad_select_x,squad_min_play_x,squad_max_play_x,ui_shirt_specific_x,sub_positions_locked_x,element_count_x,id_y,plural_name_y,plural_name_short_y,position_name,singular_name_short_y,squad_select_y,squad_min_play_y,squad_max_play_y,ui_shirt_specific_y,sub_positions_locked_y,element_count_y
0,100.0,100.0,58822,0,0,-3,3,0,2,0.5,0.5,0,Cédric,0.0,1,False,,2022-09-16T13:30:06.551108Z,42,58822.jpg,1.0,Alves Soares,0.1,False,None,a,1,3,2,6437,4,26365,9,0.0,0.5,Cédric,28,0,0,0,0,0,0,0,0,0,0,0,5,2.4,1.6,0.0,0.4,0,0.00000,0.00979,0.00979,0.04350,436,161,395,143,492,199,442,162,NaN,,NaN,,NaN,,0.00000,0.00,0.03147,0.03147,0.13982,0.00,549,158,386,151,390,146,400,148,0.00000,0.00000,3,0,None,1,0,Arsenal,0,0,0,ARS,4,None,False,0,1220,1270,1240,1250,1200,1270,1,2,Defenders,DEF,Defender,DEF,5,3,5,False,[],236,2,Defenders,DEF,Defender,DEF,5,3,5,False,[],236
1,NaN,NaN,156074,0,0,-3,3,0,2,0.5,0.5,0,Rob,0.0,5,False,,None,42,156074.jpg,1.0,Holding,0.1,False,None,a,1,3,4,6493,8,16769,13,0.0,1.0,Holding,12,0,0,0,0,0,0,0,0,0,0,0,14,2.4,0.1,0.0,0.2,0,0.00000,0.00022,0.00022,0.13740,438,163,437,165,498,203,449,165,NaN,,NaN,,NaN,,0.00000,0.00,0.00165,0.00165,1.03050,0.00,557,164,391,155,393,149,419,157,0.00000,0.00000,3,0,None,1,0,Arsenal,0,0,0,ARS,4,None,False,0,1220,1270,1240,1250,1200,1270,1,2,Defenders,DEF,Defender,DEF,5,3,5,False,[],236,2,Defenders,DEF,Defender,DEF,5,3,5,False,[],236
2,100.0,100.0,192895,0,0,-3,3,0,2,1.5,1.5,0,Kieran,1.0,8,False,,2022-09-26T13:00:06.430643Z,47,192895.jpg,1.6,Tierney,0.9,False,None,a,1,3,21,71935,52,128143,70,0.2,4.5,Tierney,417,0,0,1,3,0,0,0,0,0,0,0,85,64.0,53.8,45.0,16.3,4,0.26030,0.26686,0.52716,3.03910,307,121,230,63,260,75,313,105,NaN,,NaN,,NaN,,0.05618,0.00,0.05760,0.11378,0.65592,0.65,298,51,133,41,322,117,187,67,0.86331,0.21583,3,0,None,1,0,Arsenal,0,0,0,ARS,4,None,False,0,1220,1270,1240,1250,1200,1270,1,2,Defenders,DEF,Defender,DEF,5,3,5,False,[],236,2,Defenders,DEF,Defender,DEF,5,3,5,False,[],236
3,NaN,NaN,198869,0,0,2,-2,0,2,1.5,1.5,0,Benjamin,1.0,10,False,,None,47,198869.jpg,4.5,White,14.0,False,None,a,1,3,68,1731629,1469,873870,1187,0.2,14.5,White,1240,0,2,8,11,0,0,0,3,0,0,6,319,261.2,216.2,39.0,51.6,15,0.30690,1.71361,2.02051,12.12620,84,23,64,8,277,86,114,20,NaN,,NaN,,NaN,,0.02228,0.00,0.12437,0.14665,0.88013,0.80,299,52,134,42,36,8,29,10,1.08871,0.58065,3,0,Non

# Player game week history

Now that we have some basic information for players, teams and positions. Let’s get the gameweek points from the current season.

We can do this in two ways:

1. For each gameweek GID, get all player data from https://fantasy.premierleague.com/api/event/{GID}/
2. For each player PID, get gameweek history from https://fantasy.premierleague.com/api/element-summary/{PID}/


Since we already have all players in one dataframe, let’s go with option 2 and get data on a per-player basis.

The element-summary endpoint contains three fields at the top level:

- fixtures contains upcoming fixture information
- history contains previous gameweek player scores
- history_past provides summary of previous season totals

In [31]:
# get data from 'element-summary/{PID}/' endpoint for PID=4
r = requests.get(base_url + 'element-summary/4/').json()

# show top-level fields for player summary
pprint(r, depth=1)

{'fixtures': [...], 'history': [...], 'history_past': [...]}


In [32]:
# show data for first gameweek
pprint(r['history'][0])

{'assists': 0,
 'bonus': 0,
 'bps': 0,
 'clean_sheets': 0,
 'creativity': '0.0',
 'element': 4,
 'expected_assists': '0.00000',
 'expected_goal_involvements': '0.00000',
 'expected_goals': '0.00000',
 'expected_goals_conceded': '0.00000',
 'fixture': 1,
 'goals_conceded': 0,
 'goals_scored': 0,
 'ict_index': '0.0',
 'influence': '0.0',
 'kickoff_time': '2022-08-05T19:00:00Z',
 'minutes': 0,
 'opponent_team': 7,
 'own_goals': 0,
 'penalties_missed': 0,
 'penalties_saved': 0,
 'red_cards': 0,
 'round': 1,
 'saves': 0,
 'selected': 86132,
 'starts': 0,
 'team_a_score': 2,
 'team_h_score': 0,
 'threat': '0.0',
 'total_points': 0,
 'transfers_balance': 0,
 'transfers_in': 0,
 'transfers_out': 0,
 'value': 45,
 'was_home': False,
 'yellow_cards': 0}


In [33]:
def get_season_history(player_id):
    '''get all past season info for a given player_id'''
    
    # send GET request to
    # https://fantasy.premierleague.com/api/element-summary/{PID}/
    r = requests.get(
            base_url + 'element-summary/' + str(player_id) + '/'
    ).json()
    
    # extract 'history_past' data from response into dataframe
    df = pd.json_normalize(r['history_past'])
    
    return df

# show player #1's gameweek history
get_season_history(1)[
    [
        'season_name',
        'total_points',
        'minutes',
        'goals_scored',
        'assists'
    ]
].head(10)

,season_name,total_points,minutes,goals_scored,assists
0,2015/16,86,1965,0,2
1,2016/17,102,2515,0,3
2,2017/18,85,2794,0,3
3,2018/19,52,1493,1,2
4,2019/20,61,1553,1,1
5,2020/21,28,744,0,1
6,2021/22,48,1481,1,1


In [36]:
get_season_history(1)

,season_name,element_code,start_cost,end_cost,total_points,minutes,goals_scored,assists,clean_sheets,goals_conceded,own_goals,penalties_saved,penalties_missed,yellow_cards,red_cards,saves,bonus,bps,influence,creativity,threat,ict_index,starts,expected_goals,expected_assists,expected_goal_involvements,expected_goals_conceded
0,2015/16,58822,50,47,86,1965,0,2,9,19,0,0,0,3,0,0,7,506,0.0,0.0,0.0,0.0,0,0.00000,0.00000,0.00000,0.00000
1,2016/17,58822,50,49,102,2515,0,3,11,36,0,0,0,7,0,0,11,584,591.0,648.9,155.0,139.6,0,0.00000,0.00000,0.00000,0.00000
2,2017/18,58822,50,47,85,2794,0,3,7,44,0,0,0,3,0,0,2,481,507.0,455.6,159.0,112.3,0,0.00000,0.00000,0.00000,0.00000
3,2018/19,58822,45,42,52,1493,1,2,4,31,0,0,0,4,0,0,7,273,309.0,226.5,103.0,63.9,0,0.00000,0.00000,0.00000,0.00000
4,2019/20,58822,50,48,61,1553,1,1,4,20,0,0,0,1,0,0,3,286,349.0,218.9,118.0,68.7,0,0.00000,0.00000,0.00000,0.00000
5,2020/21,58822,50,46,28,744,0,1,2,11,0,0,0,1,0,0,3,125,110.8,114.8,66.0,29.2,0,0.00000,0.00000,0.00000,0.00000
6,2021/22,58822,45,42,48,1481,1,1,3,27,0,0,0,3,0,0,3,292,318.4,327.1,111.0,75.8,0,0.00000,0.00000,0.00000,0.00000
